This notebook analyses the effect of positive earnings surprise on the price of the stock after the earnings are reported. 

In [218]:
import pandas as pd
import numpy as np
import plotly.express as px

For the purpose of this analysis a Kaggle dataset 'US historical stock prices with earnings data' is used. Since it is a PoC the data on JD.com stock is used.

In [219]:
earnings = pd.read_csv('stock_data/earnings_latest.csv')

In [220]:
jd_earnings = earnings[earnings.symbol.eq('JD')]

In [221]:
jd_earnings.sort_values(by = ['date'])

,symbol,date,qtr,eps_est,eps,release_time
77932,JD,2014-08-15,06/2014,-0.040,0.00,pre
77933,JD,2014-11-17,09/2014,0.010,0.04,pre
77934,JD,2015-03-03,12/2014,-0.010,0.01,pre
77935,JD,2015-05-08,03/2015,-0.020,-0.02,pre
77936,JD,2015-08-07,06/2015,-0.030,0.00,pre
77937,JD,2015-11-16,09/2015,0.030,0.00,pre
77938,JD,2016-03-01,12/2015,-0.020,-0.07,pre
77939,JD,2016-05-09,03/2016,-0.030,-0.02,pre
77940,JD,2016-08-10,06/2016,-0.010,0.04,pre
77941,JD,2016-11-15,09/2016,0.000,0.03,pre


In [222]:
prices = pd.read_csv('stock_data/stock_prices_latest.csv')

In [223]:
jd_prices = prices[prices.symbol.eq('JD')]

In [224]:
jd_prices.sort_values(by = ['date'])

,symbol,date,open,high,low,close,close_adjusted,volume,split_coefficient
4154781,JD,2014-05-22,21.75,22.800,20.2400,20.90,20.90,84449700,1.0
4154853,JD,2014-05-23,21.25,21.250,19.9400,20.10,20.10,20890300,1.0
4154676,JD,2014-05-27,20.28,22.980,20.2500,22.97,22.97,15338100,1.0
4154866,JD,2014-05-28,22.98,25.300,22.8000,24.78,24.78,16783000,1.0
4154586,JD,2014-05-29,25.04,25.690,24.2400,25.40,25.40,8525900,1.0
...,...,...,...,...,...,...,...,...,...
22801733,JD,2020-09-14,75.70,76.910,75.0000,76.59,76.59,7414914,1.0
22801735,JD,2020-09-15,78.23,78.415,76.5902,77.00,77.00,8105463,1.0
22801734,JD,2020-09-16,77.01,77.070,74.9500,75.09,75.09,8073989,1.0
22801738,JD,2020-09-17,73.02,74.470,72.7556,74.37,74.37,9099387,1.0


In [225]:
jd_earnings.dtypes

symbol           object
date             object
qtr              object
eps_est         float64
eps             float64
release_time     object
dtype: object

In [226]:
jd_prices.dtypes

symbol                object
date                  object
open                 float64
high                 float64
low                  float64
close                float64
close_adjusted       float64
volume                 int64
split_coefficient    float64
dtype: object

Let's left merge the prices and earnings datasets on date. 

In [227]:
merged = jd_prices.merge(jd_earnings, on='date', how='left')

In [228]:
merged.sort_values(by="date")

,symbol_x,date,open,high,low,close,close_adjusted,volume,split_coefficient,symbol_y,qtr,eps_est,eps,release_time
603,JD,2014-05-22,21.75,22.800,20.2400,20.90,20.90,84449700,1.0,NaN,NaN,NaN,NaN,NaN
675,JD,2014-05-23,21.25,21.250,19.9400,20.10,20.10,20890300,1.0,NaN,NaN,NaN,NaN,NaN
498,JD,2014-05-27,20.28,22.980,20.2500,22.97,22.97,15338100,1.0,NaN,NaN,NaN,NaN,NaN
688,JD,2014-05-28,22.98,25.300,22.8000,24.78,24.78,16783000,1.0,NaN,NaN,NaN,NaN,NaN
408,JD,2014-05-29,25.04,25.690,24.2400,25.40,25.40,8525900,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1588,JD,2020-09-14,75.70,76.910,75.0000,76.59,76.59,7414914,1.0,NaN,NaN,NaN,NaN,NaN
1590,JD,2020-09-15,78.23,78.415,76.5902,77.00,77.00,8105463,1.0,NaN,NaN,NaN,NaN,NaN
1589,JD,2020-09-16,77.01,77.070,74.9500,75.09,75.09,8073989,1.0,NaN,NaN,NaN,NaN,NaN
1593,JD,2020-09-17,73.02,74.470,72.7556,74.37,74.37,9099387,1.0,NaN,NaN,NaN,NaN,NaN


In order to see if there was an effect on the price in the period succeeding the earnings report prices for previous and next days are needed.

In [229]:
merged['open_next_day'] = merged.open.shift(-1)
merged['high_next_day'] = merged.high.shift(-1)
merged['low_next_day'] = merged.low.shift(-1)
merged['close_next_day'] = merged.close.shift(-1)
merged['volume_next_day'] = merged.volume.shift(-1)
merged['open_previous_day'] = merged.open.shift(+1)
merged['high_previous_day'] = merged.high.shift(+1)
merged['low_previous_day'] = merged.low.shift(+1)
merged['close_previous_day'] = merged.close.shift(+1)
merged['volume_previous_day'] = merged.volume.shift(+1)

Drop the columns that aren't needed for the purpose of this analysis.

In [230]:
merged_clean = merged.drop(columns = ['close_adjusted', 'split_coefficient', 'symbol_y']).copy()
merged_clean

,symbol_x,date,open,high,low,close,volume,qtr,eps_est,eps,...,open_next_day,high_next_day,low_next_day,close_next_day,volume_next_day,open_previous_day,high_previous_day,low_previous_day,close_previous_day,volume_previous_day
0,JD,2014-09-10,29.04,29.230,28.0200,28.35,7511800,NaN,NaN,NaN,...,29.15,29.150,28.3500,28.78,12349977.0,NaN,NaN,NaN,NaN,NaN
1,JD,2015-10-05,29.15,29.150,28.3500,28.78,12349977,NaN,NaN,NaN,...,28.36,28.500,27.7100,28.01,3066300.0,29.04,29.230,28.0200,28.35,7511800.0
2,JD,2014-06-27,28.36,28.500,27.7100,28.01,3066300,NaN,NaN,NaN,...,26.87,26.970,26.2650,26.84,7892442.0,29.15,29.150,28.3500,28.78,12349977.0
3,JD,2016-12-09,26.87,26.970,26.2650,26.84,7892442,NaN,NaN,NaN,...,23.16,23.505,23.1200,23.32,9133421.0,28.36,28.500,27.7100,28.01,3066300.0
4,JD,2016-06-06,23.16,23.505,23.1200,23.32,9133421,NaN,NaN,NaN,...,21.23,21.500,20.9800,21.31,8291555.0,26.87,26.970,26.2650,26.84,7892442.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1589,JD,2020-09-16,77.01,77.070,74.9500,75.09,8073989,NaN,NaN,NaN,...,78.23,78.415,76.5902,77.00,8105463.0,75.70,76.910,75.0000,76.59,7414914.0
1590,JD,2020-09-15,78.23,78.415,76.5902,77.00,8105463,NaN,NaN,NaN,...,77.13,78.170,73.5500,74.53,11810426.0,77.01,77.070,74.9500,75.09,8073989.0
1591,JD,2020-09-10,77.13,78.170,73.5500,74.53,11810426,NaN,NaN,NaN,...,76.00,78.015,75.3800,76.22,14080540.0,78.23,78.415,76.5902,77.00,8105463.0
1592,JD,2020-09-08,76.00,78.015,75.3800,76.22,14080540,NaN,NaN,NaN,...,73.02,74.470,72.7556,74.37,9099387.0,77.13,78.170,73.5500,74.53,11810426.0


Only earnings dates are needed, so let's filter out all rows that don't have a qtr. 

In [231]:
earnings_dates_data = merged_clean[pd.notnull(merged_clean['qtr'])].copy()

In [232]:
earnings_dates_data

,symbol_x,date,open,high,low,close,volume,qtr,eps_est,eps,...,open_next_day,high_next_day,low_next_day,close_next_day,volume_next_day,open_previous_day,high_previous_day,low_previous_day,close_previous_day,volume_previous_day
38,JD,2017-05-08,37.88,39.090,37.6900,38.55,31389643,03/2017,0.020,0.15,...,31.40,31.560,31.0850,31.11,5975227.0,25.900,26.070,25.5200,25.55,9844031.0
44,JD,2017-11-13,41.97,42.770,40.8500,41.34,43593708,09/2017,0.100,0.23,...,27.75,28.400,26.9100,27.32,5952836.0,26.350,26.760,26.2010,26.52,6661683.0
130,JD,2016-05-09,23.99,23.990,22.6700,23.43,45307397,03/2016,-0.030,-0.02,...,39.00,39.620,37.9800,39.22,18130801.0,33.070,33.850,32.8700,33.72,5947083.0
215,JD,2017-08-14,45.00,45.450,43.0600,44.25,36753356,06/2017,0.080,0.10,...,25.40,27.050,25.1900,26.99,16470408.0,37.870,38.000,37.1200,37.45,16348800.0
241,JD,2016-11-15,25.69,26.960,25.2900,26.41,38369212,09/2016,0.000,0.03,...,27.00,27.730,26.9600,27.28,8124615.0,23.310,23.390,22.8600,23.17,12505389.0
247,JD,2016-08-10,24.11,24.600,23.2600,23.41,33722234,06/2016,-0.010,0.04,...,24.93,25.450,24.4800,25.20,15385510.0,29.570,30.270,29.5100,29.85,7080140.0
303,JD,2015-05-08,32.96,33.850,32.1000,33.74,11417158,03/2015,-0.020,-0.02,...,33.73,34.250,33.4900,34.02,6050612.0,44.760,46.850,44.6200,45.87,15664552.0
560,JD,2017-03-02,31.74,32.470,30.4800,30.93,22176037,12/2016,-0.050,0.06,...,26.27,27.480,26.2700,26.77,10343477.0,27.000,27.120,26.0900,26.50,11741345.0
566,JD,2015-11-16,26.71,29.580,26.5308,28.81,26242713,09/2015,0.030,0.00,...,25.60,26.575,25.5500,26.50,12505496.0,39.010,39.180,38.6000,39.01,14493508.0
689,JD,2016-03-01,26.89,27.190,26.3100,26.50,17268274,12/2015,-0.020,-0.07,...,22.20,22.300,21.5100,21.68,9906981.0,22.980,25.300,22.8000,24.78,16783000.0


In [233]:
earnings_dates_data.isnull().sum()

symbol_x               0
date                   0
open                   0
high                   0
low                    0
close                  0
volume                 0
qtr                    0
eps_est                0
eps                    0
release_time           3
open_next_day          0
high_next_day          0
low_next_day           0
close_next_day         0
volume_next_day        0
open_previous_day      0
high_previous_day      0
low_previous_day       0
close_previous_day     0
volume_previous_day    0
dtype: int64

Let's calculate the earnings_surprise and earnings_surprise in % from the data. 

In [246]:
earnings_dates_data['earnings_surprise'] = earnings_dates_data.apply(lambda x: (x['eps'] - x['eps_est']), axis=1).copy()
earnings_dates_data['earnings_surprise_percent'] = earnings_dates_data.apply(lambda x: round(abs((x['eps'] - x['eps_est'])/x['eps_est']), 2)*100 if x['eps_est'] != 0 else 0, axis=1).copy()
earnings_dates_data

,symbol_x,date,open,high,low,close,volume,qtr,eps_est,eps,...,close_next_day,volume_next_day,open_previous_day,high_previous_day,low_previous_day,close_previous_day,volume_previous_day,earnings_surprise,earnings_surprise_percent,price_increase
38,JD,2017-05-08,37.88,39.090,37.6900,38.55,31389643,03/2017,0.020,0.15,...,31.11,5975227.0,25.900,26.070,25.5200,25.55,9844031.0,0.130,650.0,True
44,JD,2017-11-13,41.97,42.770,40.8500,41.34,43593708,09/2017,0.100,0.23,...,27.32,5952836.0,26.350,26.760,26.2010,26.52,6661683.0,0.130,130.0,True
130,JD,2016-05-09,23.99,23.990,22.6700,23.43,45307397,03/2016,-0.030,-0.02,...,39.22,18130801.0,33.070,33.850,32.8700,33.72,5947083.0,0.010,33.0,False
215,JD,2017-08-14,45.00,45.450,43.0600,44.25,36753356,06/2017,0.080,0.10,...,26.99,16470408.0,37.870,38.000,37.1200,37.45,16348800.0,0.020,25.0,True
241,JD,2016-11-15,25.69,26.960,25.2900,26.41,38369212,09/2016,0.000,0.03,...,27.28,8124615.0,23.310,23.390,22.8600,23.17,12505389.0,0.030,0.0,True
247,JD,2016-08-10,24.11,24.600,23.2600,23.41,33722234,06/2016,-0.010,0.04,...,25.20,15385510.0,29.570,30.270,29.5100,29.85,7080140.0,0.050,500.0,False
303,JD,2015-05-08,32.96,33.850,32.1000,33.74,11417158,03/2015,-0.020,-0.02,...,34.02,6050612.0,44.760,46.850,44.6200,45.87,15664552.0,0.000,0.0,False
560,JD,2017-03-02,31.74,32.470,30.4800,30.93,22176037,12/2016,-0.050,0.06,...,26.77,10343477.0,27.000,27.120,26.0900,26.50,11741345.0,0.110,220.0,True
566,JD,2015-11-16,26.71,29.580,26.5308,28.81,26242713,09/2015,0.030,0.00,...,26.50,12505496.0,39.010,39.180,38.6000,39.01,14493508.0,-0.030,100.0,False
689,JD,2016-03-01,26.89,27.190,26.3100,26.50,17268274,12/2015,-0.020,-0.07,...,21.68,9906981.0,22.980,25.300,22.8000,24.78,16783000.0,-0.050,250.0,True


Let's calculate a column showing if there was an increase in price or not. For that, if the earnings were announced before market opens (release_time == 'pre'), we compare the mean of high and close prices day before with the mean of open and high prices of the earnings report day;
if the earnings were announces after the market closes (release_time == 'close'), we compare the mean of close and high prices of the earnings day with the mean of open and high prices of the next day. 

In [247]:
earnings_dates_data.loc[:, 'price_increase'] = earnings_dates_data.apply(lambda x: True if x['release_time'] == 'pre' and np.mean([x['open'], x['high']]) > np.mean([x['high_previous_day'], x['close_previous_day']]) 
or x['release_time'] == 'close' and np.mean([x['open_next_day'], x['high_next_day']]) > np.mean([x['high'], x['close']]) else False, axis=1).copy()

In [248]:
earnings_dates_data

,symbol_x,date,open,high,low,close,volume,qtr,eps_est,eps,...,close_next_day,volume_next_day,open_previous_day,high_previous_day,low_previous_day,close_previous_day,volume_previous_day,earnings_surprise,earnings_surprise_percent,price_increase
38,JD,2017-05-08,37.88,39.090,37.6900,38.55,31389643,03/2017,0.020,0.15,...,31.11,5975227.0,25.900,26.070,25.5200,25.55,9844031.0,0.130,650.0,True
44,JD,2017-11-13,41.97,42.770,40.8500,41.34,43593708,09/2017,0.100,0.23,...,27.32,5952836.0,26.350,26.760,26.2010,26.52,6661683.0,0.130,130.0,True
130,JD,2016-05-09,23.99,23.990,22.6700,23.43,45307397,03/2016,-0.030,-0.02,...,39.22,18130801.0,33.070,33.850,32.8700,33.72,5947083.0,0.010,33.0,False
215,JD,2017-08-14,45.00,45.450,43.0600,44.25,36753356,06/2017,0.080,0.10,...,26.99,16470408.0,37.870,38.000,37.1200,37.45,16348800.0,0.020,25.0,True
241,JD,2016-11-15,25.69,26.960,25.2900,26.41,38369212,09/2016,0.000,0.03,...,27.28,8124615.0,23.310,23.390,22.8600,23.17,12505389.0,0.030,0.0,True
247,JD,2016-08-10,24.11,24.600,23.2600,23.41,33722234,06/2016,-0.010,0.04,...,25.20,15385510.0,29.570,30.270,29.5100,29.85,7080140.0,0.050,500.0,False
303,JD,2015-05-08,32.96,33.850,32.1000,33.74,11417158,03/2015,-0.020,-0.02,...,34.02,6050612.0,44.760,46.850,44.6200,45.87,15664552.0,0.000,0.0,False
560,JD,2017-03-02,31.74,32.470,30.4800,30.93,22176037,12/2016,-0.050,0.06,...,26.77,10343477.0,27.000,27.120,26.0900,26.50,11741345.0,0.110,220.0,True
566,JD,2015-11-16,26.71,29.580,26.5308,28.81,26242713,09/2015,0.030,0.00,...,26.50,12505496.0,39.010,39.180,38.6000,39.01,14493508.0,-0.030,100.0,False
689,JD,2016-03-01,26.89,27.190,26.3100,26.50,17268274,12/2015,-0.020,-0.07,...,21.68,9906981.0,22.980,25.300,22.8000,24.78,16783000.0,-0.050,250.0,True


We only want the cases when earnings_surprise was positive. 

In [249]:
is_positive = earnings_dates_data['earnings_surprise'].copy() > 0
earnings_dates_data_positive_surprise = earnings_dates_data[is_positive]

Let's count how many times price_increase was true and how many false in the case of positive earnings surprise.

In [250]:
counts_of_price_increase_on_positive_earnings_surprise = earnings_dates_data_positive_surprise['price_increase'].value_counts().copy()

In [251]:
counts_of_price_increase_on_positive_earnings_surprise.to_frame()

,price_increase
True,10
False,9


Finally, let's plot the number of cases when price went up after a positive earnings surprise and when it didn't. 

In [245]:
fig = px.bar(counts_of_price_increase_on_positive_earnings_surprise)
fig.show()

We can see that there is no direct correlation between short-term price increase of the stock and a positive earnings surprise. 